In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

In [3]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [4]:
from sklearn.utils import resample
believe = train[train['sentiment'] == 1]
no_belief = train[train['sentiment'] == -1]
neutral = train[train['sentiment'] == 0]
news = train[train['sentiment'] == 2]
# upsample minority
no_belief_upsampled = resample(no_belief,replace = True,n_samples = len(believe),
                              random_state = 15)
news_upsampled = resample(news,replace = True,n_samples = len(believe),
                              random_state = 15)
neutral_upsampled = resample(neutral,replace = True,n_samples = len(believe),
                              random_state = 15)
# combine majority and upsampled minority
upsampled = pd.concat([believe, no_belief_upsampled, news_upsampled, neutral_upsampled])

In [5]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [6]:
y = upsampled['sentiment']
X = upsampled['message']

In [7]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

In [8]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.25,shuffle=True, stratify=y, random_state=11)

In [9]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

In [10]:
testx = test['message']
test_vect = vectorizer.transform(testx)

In [11]:
y_pred = rfc.predict(test_vect)

In [12]:
test['sentiment'] = y_pred

In [13]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


In [14]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)